In [5]:
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.utils import get_file
import tarfile
import os
import numpy as np
import json
from collections import Counter
from keras.optimizers import SGD
import tensorflow as tf

In [2]:
# path = get_file('pet_images', 'http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz')
path = r"C:\Users\zdwxx\Downloads\Compressed\images.tar.gz"
tar = tarfile.open(path)
tar.extractall()
tar.close()
os.rename('images', 'pet_images')

c:\users\zdwxx\appdata\local\programs\python\python37\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'images' -> 'pet_images'

In [3]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [4]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [11]:
base_model = InceptionV3(weights='./.keras/models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = Dense(1024, name='dense', activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
















__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________

In [6]:
model = tf.keras.models.load_model("retraining_inception_dog_cat_pool_dense.h5")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

In [8]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=1
)

Train on 7390 samples
Epoch 1/15
7390/7390 [==============================] - 93s 13ms/sample - loss: 0.3193 - acc: 0.8972
Epoch 2/15
7390/7390 [==============================] - 78s 11ms/sample - loss: 0.2844 - acc: 0.9085
Epoch 3/15
7390/7390 [==============================] - 80s 11ms/sample - loss: 0.2314 - acc: 0.9219
Epoch 4/15
7390/7390 [==============================] - 81s 11ms/sample - loss: 0.2028 - acc: 0.9322
Epoch 5/15
7390/7390 [==============================] - 82s 11ms/sample - loss: 0.1704 - acc: 0.9398
Epoch 6/15
7390/7390 [==============================] - 83s 11ms/sample - loss: 0.1575 - acc: 0.9488
Epoch 7/15
7390/7390 [==============================] - 84s 11ms/sample - loss: 0.1336 - acc: 0.9533
Epoch 8/15
7390/7390 [==============================] - 85s 11ms/sample - loss: 0.1208 - acc: 0.9582
Epoch 9/15
7390/7390 [==============================] - 85s 12ms/sample - loss: 0.1008 - acc: 0.9654
Epoch 10/15
7390/7390 [==============================] - 86s 12ms/sam

In [9]:
model.save("./retraining_inception_dog_cat_pool_dense.h5")

In [12]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
7390/7390 [==============================] - 89s 12ms/step - loss: 3.6279 - acc: 0.0438
Epoch 2/10
7390/7390 [==============================] - 85s 11ms/step - loss: 3.4245 - acc: 0.1346
Epoch 3/10
7390/7390 [==============================] - 86s 12ms/step - loss: 3.2285 - acc: 0.2781
Epoch 4/10
7390/7390 [==============================] - 87s 12ms/step - loss: 3.0391 - acc: 0.4576
Epoch 5/10
7390/7390 [==============================] - 88s 12ms/step - loss: 2.8513 - acc: 0.5829
Epoch 6/10
7390/7390 [==============================] - 88s 12ms/step - loss: 2.6639 - acc: 0.6624
Epoch 7/10
7390/7390 [==============================] - 87s 12ms/step - loss: 2.4750 - acc: 0.7223
Epoch 8/10
7390/7390 [==============================] - 87s 12ms/step - loss: 2.2893 - acc: 0.7564
Epoch 9/10
7390/7390 [==============================] - 87s 12ms/step - loss: 2.1003 - acc: 0.7893
Epoch 10/10
7

In [14]:
model.save('zoo/09.3 retrained pet recognizer_mixed9.h5')

In [11]:
json.dump(idx_to_labels, open('zoo/09.3 pet_labels.json', 'w'))
model.save('zoo/09.3 retrained pet recognizer.h5')
